In [1]:
import pandas as pd
train = pd.read_csv("train.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)
sample_submit = pd.read_csv("sample_submit.csv",index_col=0,header=None)

In [2]:
from pycaret.classification import *
s = setup(train,target='charges')

,Description,Value
0,Session id,252
1,Target,charges
2,Target type,Multiclass
3,Original data shape,"(1600, 7)"
4,Transformed data shape,"(1600, 10)"
5,Transformed train set shape,"(1120, 10)"
6,Transformed test set shape,"(480, 10)"
7,Ordinal features,2
8,Numeric features,3
9,Categorical features,3


In [3]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8732,0.8811,0.8732,0.8673,0.8671,0.6290,0.6334,0.7660
rf,Random Forest Classifier,0.8723,0.8873,0.8723,0.8662,0.8613,0.6181,0.6272,0.2970
gbc,Gradient Boosting Classifier,0.8679,0.8827,0.8679,0.8611,0.8599,0.6126,0.6172,0.3390
ada,Ada Boost Classifier,0.8652,0.8335,0.8652,0.8571,0.8571,0.6050,0.6100,0.2630
lda,Linear Discriminant Analysis,0.8634,0.8816,0.8634,0.8626,0.8574,0.6153,0.6197,0.2420
ridge,Ridge Classifier,0.8607,0.0000,0.8607,0.8635,0.8537,0.6075,0.6130,0.2230
nb,Naive Bayes,0.8580,0.8530,0.8580,0.8562,0.8527,0.5988,0.6024,0.2260
lightgbm,Light Gradient Boosting Machine,0.8554,0.8837,0.8554,0.8445,0.8466,0.5756,0.5804,0.4100
et,Extra Trees Classifier,0.8545,0.8725,0.8545,0.8473,0.8468,0.5759,0.5815,0.3090
qda,Quadratic Discriminant Analysis,0.8259,0.8435,0.8259,0.8416,0.8258,0.5279,0.5354,0.2310


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=252, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [4]:
lr = create_model('lr')
rf = create_model('rf')
gbc = create_model('gbc')

tuned_lr = tune_model(lr)
tuned_rf = tune_model(rf)
tuned_gbc = tune_model(gbc)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8661,0.9602,0.8661,0.8430,0.8470,0.5701,0.5871
1,0.8839,0.8784,0.8839,0.8742,0.8753,0.6543,0.6594
2,0.8929,0.8914,0.8929,0.8849,0.8827,0.6752,0.6835
3,0.9107,0.9023,0.9107,0.9127,0.9114,0.7511,0.7513
4,0.8482,0.8263,0.8482,0.8567,0.8446,0.5454,0.5499
5,0.8482,0.8529,0.8482,0.8394,0.8434,0.5629,0.5638
6,0.8839,0.8460,0.8839,0.8849,0.8822,0.6648,0.6664
7,0.8661,0.8764,0.8661,0.8504,0.8562,0.5928,0.5984
8,0.8661,0.8804,0.8661,0.8668,0.8663,0.6337,0.6340


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8929,0.9647,0.8929,0.8869,0.8766,0.6564,0.6773
1,0.8750,0.8831,0.8750,0.8617,0.8575,0.6146,0.6272
2,0.9018,0.9007,0.9018,0.9101,0.8853,0.6854,0.7090
3,0.8839,0.9373,0.8839,0.8810,0.8823,0.6713,0.6716
4,0.8304,0.8228,0.8304,0.8299,0.8218,0.4733,0.4806
5,0.8482,0.8676,0.8482,0.8441,0.8461,0.5702,0.5704
6,0.8482,0.8336,0.8482,0.8389,0.8414,0.5462,0.5496
7,0.8929,0.9081,0.8929,0.8813,0.8833,0.6749,0.6822
8,0.8750,0.8682,0.8750,0.8722,0.8674,0.6531,0.6567


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.9265,0.8571,0.8374,0.8429,0.5669,0.5739
1,0.8839,0.8774,0.8839,0.8777,0.8784,0.6603,0.6633
2,0.8750,0.9251,0.8750,0.8608,0.8570,0.6146,0.6272
3,0.8750,0.9148,0.8750,0.8750,0.8740,0.6525,0.6531
4,0.8304,0.8169,0.8304,0.8321,0.8186,0.4629,0.4733
5,0.8661,0.8521,0.8661,0.8536,0.8585,0.6004,0.6039
6,0.8571,0.8621,0.8571,0.8629,0.8577,0.5944,0.5955
7,0.9018,0.8661,0.9018,0.8968,0.8988,0.7172,0.7183
8,0.8661,0.9092,0.8661,0.8603,0.8612,0.6280,0.6292


...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9018,0.9645,0.9018,0.8947,0.8939,0.7020,0.7095
1,0.8839,0.8761,0.8839,0.8731,0.8701,0.6417,0.6533
2,0.9018,0.9110,0.9018,0.9115,0.8859,0.6915,0.7111
3,0.8929,0.9181,0.8929,0.8934,0.8915,0.6972,0.6984
4,0.8482,0.8008,0.8482,0.8444,0.8375,0.5199,0.5308
5,0.8661,0.8607,0.8661,0.8617,0.8635,0.6140,0.6148
6,0.8750,0.8381,0.8750,0.8658,0.8689,0.6267,0.6303
7,0.8839,0.8992,0.8839,0.8710,0.8749,0.6475,0.6538
8,0.8571,0.9020,0.8571,0.8457,0.8488,0.5965,0.5983


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [5]:
blender_specific = blend_models(estimator_list = [tuned_lr,tuned_rf,tuned_gbc], method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9018,0.9729,0.9018,0.8975,0.8896,0.6909,0.7069
1,0.8839,0.8833,0.8839,0.8731,0.8701,0.6417,0.6533
2,0.9018,0.9086,0.9018,0.8990,0.8905,0.6969,0.7094
3,0.9018,0.9172,0.9018,0.9003,0.9003,0.7221,0.7227
4,0.8304,0.8026,0.8304,0.8406,0.8234,0.4822,0.4886
5,0.8661,0.8607,0.8661,0.8585,0.8619,0.6143,0.6153
6,0.8750,0.8428,0.8750,0.8658,0.8689,0.6267,0.6303
7,0.8839,0.8886,0.8839,0.8710,0.8749,0.6475,0.6538
8,0.8661,0.8952,0.8661,0.8616,0.8609,0.6342,0.6361


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
predictions = predict_model(blender_specific,data=test)

In [7]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 13 to 1995
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               400 non-null    int8    
 1   sex               400 non-null    category
 2   bmi               400 non-null    float32 
 3   children          400 non-null    int8    
 4   smoker            400 non-null    category
 5   region            400 non-null    category
 6   prediction_label  400 non-null    int32   
 7   prediction_score  400 non-null    float64 
dtypes: category(3), float32(1), float64(1), int32(1), int8(2)
memory usage: 27.9 KB


In [8]:
predictions.head()

,age,sex,bmi,children,smoker,region,prediction_label,prediction_score
id,,,,,,,,
13,19,male,24.365177,1,no,northeast,0,0.9746
23,59,male,33.997765,0,yes,northeast,2,0.6452
27,42,female,29.283449,0,no,southwest,0,0.9158
28,30,male,24.903725,1,no,northwest,0,0.9614
29,41,female,29.644537,0,no,southwest,0,0.9255


In [9]:
sample_submit = pd.read_csv("sample_submit.csv",header=None, names=['id','charges'])

In [10]:
sample_submit = pd.merge(sample_submit,predictions['prediction_label'],on='id')

In [11]:
sample_submit = sample_submit.drop('charges',axis=1)

In [12]:
pd.DataFrame(sample_submit).to_csv('Pycaret_ensemble.csv', index=False, header=False)